<h1>MODELLING SWISS VOTATION OUTCOMES</h1>
<h3>Coursera Capstone Report for the IBM Professional Certificate in Data Science</h3><br>
Juliane Klatt
<hr>
<h2>Table of Contents</h2>
<ol>
    <li><a href="#intro">Introduction</a></li>
    <li><a href="#data">Data</a></li>
        <ol>
            <li><a href="#pleb">Swiss Plebiscite Data</a></li>
            <li><a href="#mun">List of Swiss Municipalities</a></li>
            <li><a href="#four">Foursquare Location Data</a></li>
        </ol>
    <li><a href="#meth">Methodology</a></li>
    <li><a href="#ana">Analysis</a></li>
    <li><a href="#res">Results</a></li>
    <li><a href="#con">Conclusion</a></li>
    <li><a href="#ref">References</a></li>
</ol>

<a id='intro'></a>
<hr>
<h2>1. Introduction</h2>

<p>In Switzerland, plebiscites at federal, cantonal, and municipal level are a central feature of political life. It is not the government's choice whether or when a referendum is held, but it is a legal procedure regulated by the Swiss constitution. There are two types of referendums: optional and mandatory referendums. Any federal law, certain other federal resolutions, and international treaties that are ongoing in nature, or any change to Swiss law may be subject to an optional referendum. Such referendum takes place if at least 50,000 people or eight cantons have petitioned to do so within 100 days. Mandatory referendums take place on any amendment to the constitution and on any joining of a multinational community or organization for collective security. Constitutional amendments are proposed by the parliament or by the cantons or by federal popular initiative.</p>
<p>In this project we will investigate whether canton-level outcomes of Swiss federal plebiscites are correlated with the type and quantity of venues present in a canton. Once correlations have been identified, several classification models will be built and compared in their ability to correctly predict swiss votation outcomes by canton based on the canton's venue characteristics.</p>

<a id='data'></a>
<hr>
<h2>2. Data</h2>
<p>
In order to meet the above objectives, several data sources are required. We need to infer canton-level outcomes of past referencdums, and we need list of venues per canton.
<ul>
    <li><a href="https://www.bk.admin.ch/bk/de/home/politische-rechte/volksabstimmungen.html">Swiss plebiscite data</a><br>
        Swiss authorities provide the canton-level outcomes of federal plebiscites since 1848
    </li>
    <li><a href="https://en.wikipedia.org/wiki/List_of_municipalities_of_Switzerland">List of Swiss municipalities</a><br>
        Wikipedia provides a list of Swiss municipalities by canton 
    </li>
    <li><a href="https://foursquare.com/">Foursquare location data</a><br>
        Foursquare provides venue data for municipalities of interest
    </li>
</ul>
In the following we will compile the aborementioned data in dataframes and quickly characterize their properties.
</p>

<a id='pleb'></a>
<h3>2.A. Swiss Plebiscite Data</h3>
<p>The canton-level outcomes of past Swiss federal plebiscites since 1848 are published by Swiss authorities on <a href="https://www.bk.admin.ch/ch/d/pore/va/vab_2_2_4_1.html">www.bk.admin.ch</a>. We will pull websites by means of the <b>requests</b> package and scrape their respective content employing the <b>BeautifulSoup</b> package.

In [64]:
import requests

plebiscite_link = 'https://www.bk.admin.ch/ch/d/pore/va/vab_2_2_4_1_gesamt.html'
plebiscite_page = requests.get(plebiscite_link)
plebiscite_doc = plebiscite_page.text

In [117]:
from bs4 import BeautifulSoup

plebiscite_soup = BeautifulSoup(html_doc,'html.parser')
plebiscite_table = plebiscite_soup.find('table')
plebiscite_rows = plebiscite_table.findAll('tr')
plebiscite_rows = plebiscite_rows[1:]
plebiscite_rows = plebiscite_rows[::-1]

In [188]:
from tqdm import tqdm
import pandas as pd

plebiscite_df = pd.DataFrame(index=range(1000))

i=0
previous_link = ''
for r in tqdm(plebiscite_rows[5:]):
    r_link  = 'https://www.bk.admin.ch/ch/d/pore/va/'+r.findAll('a',href=True)[0]['href']
    if r_link != previous_link:
        r_page  = requests.get(r_link)
        r_doc   = r_page.text
        r_soup  = BeautifulSoup(r_doc,'html.parser')
        if len(r_soup.findAll('article')) != 0:
            for href in r_soup.findAll('article')[0].findAll('a',href=True):
                if href['href'].startswith("./can"):
                    p_link  = r_link[:-11]+href['href'][1:]
                    p_page  = requests.get(p_link)
                    p_doc   = p_page.text
                    p_soup  = BeautifulSoup(p_doc,'html.parser')
                    entries = p_soup.find('table').find('tbody').findAll('tr')
                    for e in entries[:-1]:
                        canton = e.findAll('td')[0].string
                        if   canton == 'ZÃ¼rich':         canton = 'Zuerich'
                        elif canton == 'GraubÃ¼nden':     canton = 'Graubuenden'
                        elif canton == 'MilitÃ¤rschulen': canton = 'Militaerschulen'
                        yes    = int(''.join(e.findAll('td')[4].string.split("'")))
                        no     = int(''.join(e.findAll('td')[5].string.split("'")))
                        if yes > no : vote = 1
                        else:         vote = 0
                        if canton in plebiscite_df.columns:
                            plebiscite_df.loc[i,canton] = vote
                        else:
                            plebiscite_df[canton] = ''
                            plebiscite_df.loc[i,canton] = vote
                    i += 1
        previous_link = r_link

plebiscite_df = plebiscite_df[:i]
plebiscite_df = plebiscite_df.drop('Militaerschulen',axis=1)

100%|██████████| 633/633 [14:22<00:00,  1.36s/it]


In [189]:
plebiscite_df.head()

,Zuerich,Bern,Luzern,Uri,Schwyz,Obwalden,Nidwalden,Glarus,Zug,Freiburg,...,St. Gallen,Graubuenden,Aargau,Thurgau,Tessin,Waadt,Wallis,Neuenburg,Genf,Jura
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,1,1,1,1,0,1
4,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,1,1,0,1


<a id='mun'></a>
<h3>2.B. Swiss Municipalities by Canton and Georeferences</h3>
<a href="https://en.wikipedia.org/wiki/List_of_municipalities_of_Switzerland">Source</a>

<a id='four'></a>
<h3>2.C. Foursquare Location Data</h3>
<a href="https://foursquare.com/">Source</a>

<a id='meth'></a>
<hr>
<h2>3. Methodology</h2>

<a id='ana'></a>
<hr>
<h2>4. Analysis</h2>

<a id='res'></a>
<hr>
<h2>5. Results</h2>

<a id='con'></a>
<hr>
<h2>6. Conclusion</h2>

<a id='ref'></a>
<hr>
<h2>7. References</h2>